# Transcribe audio and get job data with Amazon Transcribe using an AWS SDK
- [Link](https://docs.aws.amazon.com/transcribe/latest/dg/example_transcribe_Scenario_GettingStartedTranscriptionJobs_section.html)

### 필수 조건
- 이 노트북을 실행하는 Role 에 아래의 권한이 있어야 합니다.
    - AmazonTranscribeFullAccess
    - AmazonS3FullAccess

## 데이타 업로드

In [11]:
! aws s3 cp voice-examples.wav s3://sagemaker-us-west-2-057716757052/2024-08-LG-GenAI-PoC/

upload: ./voice-examples.wav to s3://sagemaker-us-west-2-057716757052/2024-08-LG-GenAI-PoC/voice-examples.wav


In [12]:
! aws s3 ls s3://sagemaker-us-west-2-057716757052/2024-08-LG-GenAI-PoC/


2024-07-30 14:22:48    3406336 voice-examples.wav


## 스크립트 실행

In [16]:
import time
import boto3


def transcribe_file(job_name, file_uri, transcribe_client):
    transcribe_client.start_transcription_job(
        TranscriptionJobName=job_name,
        Media={"MediaFileUri": file_uri},
        MediaFormat="wav",
        LanguageCode="ko-KR",
    )

    max_tries = 60
    while max_tries > 0:
        max_tries -= 1
        job = transcribe_client.get_transcription_job(TranscriptionJobName=job_name)
        job_status = job["TranscriptionJob"]["TranscriptionJobStatus"]
        if job_status in ["COMPLETED", "FAILED"]:
            print(f"Job {job_name} is {job_status}.")
            if job_status == "COMPLETED":
                print(
                    f"Download the transcript from\n"
                    f"\t{job['TranscriptionJob']['Transcript']['TranscriptFileUri']}."
                )
            break
        else:
            print(f"Waiting for {job_name}. Current status is {job_status}.")
        time.sleep(10)


def main():
    transcribe_client = boto3.client("transcribe")
    file_uri = "s3://sagemaker-us-west-2-057716757052/2024-08-LG-GenAI-PoC/voice-examples.wav"
    transcribe_file("Example-job", file_uri, transcribe_client)


if __name__ == "__main__":
    main()



Waiting for Example-job. Current status is IN_PROGRESS.
Waiting for Example-job. Current status is IN_PROGRESS.
Waiting for Example-job. Current status is IN_PROGRESS.
Waiting for Example-job. Current status is IN_PROGRESS.
Waiting for Example-job. Current status is IN_PROGRESS.
Job Example-job is COMPLETED.
Download the transcript from
	https://s3.us-west-2.amazonaws.com/aws-transcribe-us-west-2-prod/057716757052/Example-job/64dac52e-30ed-4ae4-a062-9c655113ae5e/asrOutput.json?X-Amz-Security-Token=IQoJb3JpZ2luX2VjEF4aCXVzLXdlc3QtMiJIMEYCIQC4SWPqYeBsKuad2Tl3wGoGuib01PPFAvs76IA%2FbeQoWwIhAKRt5H6TQmm2Jc5Llniu%2BzwgW0tbdMAzYD9EghuqgeN%2FKrEFCEcQBBoMMDgwMjQ4MzIyMjA2IgxI0RW2HrmbGDExbU4qjgUW6jjKq%2FCM7Re5Hsxsk13SaIg7eCZztJoiX1wuJFBEO5hbWZMm2%2FzHX5JBFlXPsJDxPrcligCxgpInSdPTrcq8oA8bWOM4KH7sf3t0sTyMFL13gJYherpvYVIRUL7wS1mw%2FY423l1AmL5t7D2MtwTyIxT2mouigjVU853LkTNryhqEIVOe1uEh7wmufPwGcMeyEb2FN%2B8f6weY1z6WeBs7Yo3O16f9ejlmBRG6U%2B6I6P0vos2ei68KHtWKSTk4EU5zVcy5Mryn%2FnYHgjFOHQoxbKwZ%2B4iRQx6yX1Hv5